## Multimodal vs. Baseline Models
- Notice this notebook runs with the sample data for you to play with the code. Please change the configurations and use the full data to get better results.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from sklearn.model_selection import GroupKFold
from utils.config import Config
from utils.loss import KLDivLossWithLogits, CrossEntropyLossWithLogits
from eeg_dataset.data_loader import EEGDataset, EEGSpecDataset, EEGWaveDataset
from eeg_dataset.data_utils import get_fold_dls
from exp.experiment import Experiment

In [2]:
!pip install pytorchtools

In [3]:
if not os.path.exists(Config.output_dir):
    os.makedirs(Config.output_dir)

In [4]:
pl.seed_everything(Config.seed, workers=True)

42

### Load the data

In [5]:
Config.data_root = "/kaggle/input/hba-sampled-data/"
Config.PRE_LOADED_SPECTOGRAMS = "/kaggle/input/hba-sampled-data/eeg_specs.npy"
Config.PRE_LOADED_EEGS = "/kaggle/input/hba-sampled-data/eegs.npy"

In [6]:
"""The main function to run the experiment."""

# Load the data
df = pd.read_csv(f"{Config.data_root}train_300_patients.csv")
TARGETS = Config.TARGETS

train_df = df.groupby("eeg_id")[
    ["spectrogram_id", "spectrogram_label_offset_seconds"]
].agg({"spectrogram_id": "first", "spectrogram_label_offset_seconds": "min"})
train_df.columns = ["spectogram_id", "min"]

aux = df.groupby("eeg_id")[["spectrogram_id", "spectrogram_label_offset_seconds"]].agg(
    {"spectrogram_label_offset_seconds": "max"}
)
train_df["max"] = aux

aux = df.groupby("eeg_id")[["patient_id"]].agg("first")
train_df["patient_id"] = aux

aux = df.groupby("eeg_id")[TARGETS].agg("sum")
for label in TARGETS:
    train_df[label] = aux[label].values

y_data = train_df[TARGETS].values
y_data = y_data / y_data.sum(axis=1, keepdims=True)
train_df[TARGETS] = y_data
aux = df.groupby("eeg_id")[["expert_consensus"]].agg("first")
train_df["target"] = aux
train = train_df.reset_index()
all_spectrograms = np.load(Config.PRE_LOADED_SPECTOGRAMS, allow_pickle=True).item()
all_eegs = np.load(Config.PRE_LOADED_EEGS, allow_pickle=True).item()

# KFold split
gkf = GroupKFold(n_splits=5)
train["fold"] = 0
for fold, (_, val_idx) in enumerate(gkf.split(train, train.target, train.patient_id)):
    train.loc[val_idx, "fold"] = fold

kfold_data = {"all": [], "spec": [], "wave": []}
for fold_id in range(5):
    df_train = train[train.fold != fold_id]
    df_valid = train[train.fold == fold_id]
    kfold_data["all"].append(
        get_fold_dls(df_train, df_valid, all_eegs, all_spectrograms, EEGDataset)
    )
    kfold_data["spec"].append(
        get_fold_dls(df_train, df_valid, all_eegs, all_spectrograms, EEGSpecDataset)
    )
    kfold_data["wave"].append(
        get_fold_dls(df_train, df_valid, all_eegs, all_spectrograms, EEGWaveDataset)
    )

In [7]:
len(kfold_data["all"])

5

### Build up the EEGWaveNet and EEGSpecNet model

In [8]:
import torch

torch.set_float32_matmul_precision("high")

In [9]:
loss_name = "KLDiv"
model_name = "EEGModel"
if loss_name == "CrossEntropy":
    loss_cls = CrossEntropyLossWithLogits
else:
    loss_cls = KLDivLossWithLogits

# run task
if model_name == "EEGModel":
    exp = Experiment(kfold_data["all"], train, model_name, loss_cls)
elif model_name == "EEGWaveNet":
    exp = Experiment(kfold_data["wave"], train, model_name, loss_cls)
elif model_name == "EEGSpecNet":
    exp = Experiment(kfold_data["spec"], train, model_name, loss_cls)

config = Config

print(f">>>>>>>> start training: {model_name} >>>>>>>>")
exp.kfold_train(config)

>>>>>>>> start training: EEGModel >>>>>>>>
500


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Running trainer.fit


2024-04-06 03:18:37.923066: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-06 03:18:37.923164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-06 03:18:38.039874: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

100%|██████████| 16/16 [00:53<00:00,  3.33s/it]


OOF Score for solution = 0.779089081497595
